Defining the Used functions.


In [ ]:
import torch
from torch import nn
from torch.nn import Linear
import torch.nn.functional as F
!pip install torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import torchvision.transforms as transforms

# Define the shape of the signature image
C, H, W = 1, 224, 224  # Example values, update as needed
D = 64  # Dimension of each block
k=5 # no. of nearset neighbours.
#  we adopt AdamW optimizer with lr = 0.01

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, C, D):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(in_channels=C, out_channels=D, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x


def build_graph(features, k):
    """
    Builds an undirected graph from the signature image features using the K nearest neighbors algorithm.

    Args:
        features (np.ndarray): 2D array of shape (num_vertices, feature_dim) containing the feature vectors for each vertex.
        k (int): Number of nearest neighbors to consider for each vertex.

    Returns:
        np.ndarray: Edge index array of shape (2, num_edges) representing the undirected edges in the graph.
    """
    num_vertices = features.shape[0]    #  ----> N
    features= features.detach().numpy() # ----> D
    # feature_vectors = features.view(N, D)

    # Find the K nearest neighbors for each vertex
    neigh = NearestNeighbors(n_neighbors=k+1, metric='euclidean')
    neigh.fit(features)
    knn_indices = neigh.kneighbors(features, return_distance=False)[:, 1:]  # Exclude self-connections

    # Construct the edge index array
    edges = []
    for i in range(num_vertices):
        for j in knn_indices[i]:
            edges.append([i, j])
            edges.append([j, i])  # Add reverse edge to make the graph undirected

    edge_index = np.array(edges).T

    return edge_index

class SigGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(SigGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x


def euclidean_distance(x1, x2):
    """
    Calculates the Euclidean distance between two tensors.

    Args:
        x1 (torch.Tensor): First tensor.
        x2 (torch.Tensor): Second tensor.

    Returns:
        torch.Tensor: Euclidean distance between the two tensors.
    """
    return torch.sqrt(torch.sum((x1 - x2) ** 2, dim=-1))

def margin_focal_loss(y_true, gr1, gr2, alpha=10, a=0.3, b=0.6, ma=0.3, mb=0.9):
    """
    Calculates the margin-based focal loss as described in the image.

    Args:
        y_true (torch.Tensor): Ground-truth labels.
        y_pred (torch.Tensor): Predicted distances between signature representations.
        alpha (float): Scalar to adjust the loss weight.
        a (float): Margin for avoiding overfitting during training for genuine-genuine pairs.
        b (float): Margin for avoiding overfitting during training for genuine-forgery pairs.
        mb (float): Margin for avoiding overfitting during training for forgery-forgery pairs.

    Returns:
        torch.Tensor: Margin-based focal loss.
    """
    d = euclidean_distance(gr1, gr2)
    loss = y_true * torch.sigmoid(alpha * (d - a)) * torch.max(d - ma, dim=0)[0] ** 2 + \
       (1 - y_true) * torch.sigmoid(alpha * (b - d)) * torch.max(mb - d, dim=0)[0] ** 2
    return loss.mean()


Running the model for 1st Image.

In [ ]:
# Load the png image
image_path = "/content/sign1.png" #koi bhi sign img download karke.
pil_image = Image.open(image_path)

# Define any necessary image transformations
transform = transforms.Compose([
    transforms.Resize((H, W)),  # Resize the image to the desired dimensions (H, W)
    transforms.Grayscale(),
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
])

# Apply the transformations to the loaded image
sign1 = transform(pil_image).unsqueeze(0)

# Initialize the CNN model
cnn_model = CNN(1, D)

# Apply the CNN model to the signature image
feature_vector1 = cnn_model(sign1) # dimensions (1,64,224,224)

# Reshape the feature vectors to (N, D) tensor
N = feature_vector1.size(2) * feature_vector1.size(3)
# N=h*w.

feature_vector1 = feature_vector1.view(-1, D) # now it has a size of N*D.
# feature_vectors = feature_vectors.view(1, D, N).permute(0, 2, 1)
# The -1 here means that the size of this dimension is inferred so that the total number of elements remains the same. Basically,
#  it means “as many rows as necessary.”

# D is the number of output channels, so the new shape is (batch_size * height * width, D) = (N, D).

# Build the graph from the features
edge_index1 = build_graph(feature_vector1, k)
# (2, 40960) edge index shape

# Create a Data object
edge_index = torch.tensor(edge_index1, dtype=torch.long)

# data = ...  PyTorch Geometric Data object with x and edge_index
data1 = Data(x=feature_vector1, edge_index=edge_index)

model = SigGCN(in_channels=64, hidden_channels=128, out_channels=256)

# Pass the graph data through the model
out1 = model(data1)
print(out1.shape)

torch.Size([50176, 256])


Running the model for 2nd Signature Image.

In [ ]:
# Load the png image
image_path1 = "/content/sign2.png" #koi bhi sign img download karke.
pil_image1 = Image.open(image_path1)

# Define any necessary image transformations
transform = transforms.Compose([
    transforms.Resize((H, W)),  # Resize the image to the desired dimensions (H, W)
    transforms.Grayscale(),
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
])

# Apply the transformations to the loaded image
sign2 = transform(pil_image1).unsqueeze(0)

cnn_model = CNN(1, D)

# Apply the CNN model to the signature image
feature_vector2 = cnn_model(sign2)

# Reshape the feature vectors to (N, D) tensor
N = feature_vector2.size(2) * feature_vector2.size(3)
feature_vector2 = feature_vector2.view(-1, D) # now it has a size of N*D.

# features= ... # 2D array of shape (num_vertices, feature_dim) containing the signature image features

# Build the graph from the features
edge_index2 = build_graph(feature_vector2, k)

# Create a Data object
edge_index = torch.tensor(edge_index1, dtype=torch.long)
data2 = Data(x=feature_vector2, edge_index=edge_index)

out2 = model(data2)  # Graph representation of the second signature

print(out2.shape)

torch.Size([50176, 256])


Generating final graph representation and calculating loss value.

In [ ]:
print(out1.shape)
print(out2.shape)
# torch.Size([50176, 256])
# torch.Size([50176, 256])

# # Calculate the loss
y_true = 0 # Ground-truth label (0 for genuine-genuine pair, 1 for genuine-forgery pair)

# alpha=10, a=0.3, b=0.6, ma=0.3, mb=0.9 - intial values, train them and get optimal values of parameters, using lr=0.01
loss = margin_focal_loss(y_true, out1, out2, 10, 0.3, 0.6, 0.3, 0.9)

# # Compare the distance with a threshold for verification
distance = euclidean_distance(out1, out2)
threshold = 0.5 # Threshold value for verification

print(distance)
# print(distance.shape)
print(loss)

print('Distance(loss) between two signatures is: ')
distance_val= float(loss.item())
print(distance_val)
is_verified = (distance_val < threshold)
print('Is Original ? ')
print(is_verified)
if(is_verified):
  print("-->Original Pair")
else:
  print("-->Forged Pair.")

torch.Size([50176, 256])
torch.Size([50176, 256])
tensor([23.0807, 23.2121, 23.0211,  ...,  0.9195,  0.8767,  1.3473],
       grad_fn=<SqrtBackward0>)
tensor(0.0421, grad_fn=<MeanBackward0>)
Distance(loss) between two signatures is: 
0.04211956262588501
Is Original ? 
True
-->Original Pair
